## Imports and Setup

In [1]:
## Project Imports
import gspread
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [2]:
pres_data = pd.read_excel(r"C:\Users\gr0581vr\Desktop\homework\2022-2024-elections-main\full-data.xlsx", sheet_name=0)
sen_data = pd.read_excel(r"C:\Users\gr0581vr\Desktop\homework\2022-2024-elections-main\full-data.xlsx", sheet_name=1)

In [3]:
cwd = os.getcwd() ## Needed for writing csvs 
print(cwd)

C:\Users\gr0581vr\Desktop\homework\2022-2024-elections-main


In [4]:
## First we form predictions for presidential data
## We will use 2016 results as a training set, this will involve using all 
## features above except for 2020 population and 2020 polling info columns 

le = LabelEncoder()
region_enc = le.fit_transform(pres_data['Region'])
pres_data["Region"] = region_enc
sen_data["Region"] = region_enc

In [5]:
noninc_cols_train = ['State','2020 Population','recent-five-polling-avg-20','polling-party-lead-20','pres2016','pres2020']
pres_train_X = pres_data.drop(noninc_cols_train, axis=1)
pres_train_y = pres_data[['pres2016']]

## Model Creation

Models here are: logistic regression with no penalty, logistic regression models with l1 and l2 penalization terms, random forests, xgboost.

Note for logistic regression we employ a standard scaling as necessary when applying regularization terms. We keep the standardization even for no-penalty (regular) logistic regression just for the sake of completion. No standardization is done for 

In [6]:
## Logistic Regression with Scaling
scaler = StandardScaler()
lr = LogisticRegression(penalty='none', solver = 'saga')
lrmodel = Pipeline([('standardize', scaler), ('log_reg', lr)])
lrmodel.fit(pres_train_X,pres_train_y.values.ravel())

## L1-Penalty Logistic Regression ("LASSO-like" logistic regression)
lassor = LogisticRegression(penalty='l1', solver = 'saga')
lassomodel = Pipeline([('standardize', scaler), ('lasso_log_reg', lassor)])
lassomodel.fit(pres_train_X,pres_train_y.values.ravel())

## L2-Penalty Logistic Regression
ridger = LogisticRegression()
ridgemodel = Pipeline([('standardize', scaler), ('ridge_log_reg', ridger)])
ridgemodel.fit(pres_train_X,pres_train_y.values.ravel())

## Elastic Net Logistic Regression (l1_ratio = 0.2)
elo2r = LogisticRegression(penalty='elasticnet', solver = 'saga', l1_ratio = 0.2)
elo2model = Pipeline([('standardize', scaler), ('elo2_reg', elo2r)])
elo2model.fit(pres_train_X,pres_train_y.values.ravel())

## Elastic Net Logistic Regression (l1_ratio = 0.4)
elo4r = LogisticRegression(penalty='elasticnet', solver = 'saga', l1_ratio = 0.4)
elo4model = Pipeline([('standardize', scaler), ('elo4_reg', elo4r)])
elo4model.fit(pres_train_X,pres_train_y.values.ravel())

## Elastic Net Logistic Regression (l1_ratio = 0.6)
elo6r = LogisticRegression(penalty='elasticnet', solver = 'saga', l1_ratio = 0.6)
elo6model = Pipeline([('standardize', scaler), ('elo6_reg', elo6r)])
elo6model.fit(pres_train_X,pres_train_y.values.ravel())

## Elastic Net Logistic Regression (l1_ratio = 0.8)
elo8r = LogisticRegression(penalty='elasticnet', solver = 'saga', l1_ratio = 0.8)
elo8model = Pipeline([('standardize', scaler), ('elo8_reg', elo8r)])
elo8model.fit(pres_train_X,pres_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anacond

Pipeline(steps=[('standardize', StandardScaler()),
                ('elo8_reg',
                 LogisticRegression(l1_ratio=0.8, penalty='elasticnet',
                                    solver='saga'))])

## Some Inference Info (Coefficients)

In [7]:
## Coefficients organized from least important to most important
print("Logistic Regression - No Penalty")
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print("LASSO Regression")
print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("Ridge Regression")
print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.2 Logistic Regression")
print(np.sort(elo2r.coef_))
print(np.argsort(np.array(lr.coef_)))

print("EL-0.4 Logistic Regression")
print(np.sort(elo4r.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("EL-0.6 Logistic Regression")
print(np.sort(elo6r.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.8 Logistic Regression")
print(np.sort(elo8r.coef_))
print(np.argsort(np.array(lr.coef_)))

Logistic Regression - No Penalty
[[-0.616832   -0.55669398 -0.5116188  -0.43863403 -0.32843077 -0.2900192
  -0.27718827 -0.16079679 -0.14881155 -0.08675923  0.05993003  0.07173515
   0.16219093  0.18767925  0.19317982  0.20133463  0.26538569  0.29778238
   0.32005358  0.36882285  0.40625299  0.44469243  0.4487799   0.46125644
   0.47341182  0.50668156  0.53205522  0.56634678  0.69060583  0.72805877
   1.95479828]]
[[ 9 24  1 11 21  8  6 17 10 15 23  0 22 29  2  3 18 19 12 20 27  4 26 16
   5  7 25 14 13 28 30]]
LASSO Regression
[[-0.02513214  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.12160459  0.2028711   0.26648594  0.6664251
   2.42609907]]
[[ 9  0 27 26 25 24 23 22 21 20 19 18 17 16 29 15 12 11 10  8  6  4  3  2
   1 14  7  5 13 28 30]]
Ridge Regressi

## Creating a Test Set

Test set here is similar features to the training data, however we make the following changes:

*   Obviously remove any 2016 related info, so we remove 2010 population counts and the 2016 presidential polling averages.
*   From the training data, we fit the model with using 2000 presidential election results, but instead we will use 



In [8]:
noninc_cols_test = ['State','2010 Population','recent-five-polling-avg-16','polling-party-lead-16','pres2000','pres2020']
pres_test_X = pres_data.drop(noninc_cols_test, axis=1)
pres_test_y = pres_data[['pres2020']]

## Move 2016 results to position where 2012 results
## Here we use 2004, 2008, 2012, 2016 results as predictors towards 2020 results
## Since 2000 was removed we have to rearrange columns here
col_move = pres_test_X.pop("pres2016")
pres_test_X.insert(2, "pres2016", col_move)

## Rename columns just so the models can run
## A little bit hacky but we turn the 2000 results into 2016 results
dict = {'2020 Population': '2010 Population',
        'polling-party-lead-20': 'polling-party-lead-16',
        'pres2016': 'pres2000',
        'recent-five-polling-avg-20': 'recent-five-polling-avg-16',}
 
# call rename () method
pres_test_X.rename(columns=dict,inplace=True)

In [9]:
## Form predictions over all models (omitted logistic regression/ridge/lasso models, see other notebook)
elo2preds = elo2model.predict(pres_test_X)
elo4preds = elo4model.predict(pres_test_X)
elo6preds = elo6model.predict(pres_test_X)
elo8preds = elo8model.predict(pres_test_X)

In [10]:
## Notably, all the classification results are the same
print(elo2preds)
print(elo4preds)
print(elo6preds)
print(elo8preds)

[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]


## Senate Predictions

Here we form predictions for the US senate for the upcoming 2022 Midterm Elections.

As a note, the preprocessing here for the senate data is altered. We have a set of 13 "competitive" states that are considered competitive based off [2022 US Senate Election Ratings](https://en.wikipedia.org/wiki/2022_United_States_Senate_elections#Predictions). These will form our test set,and the training set will be over other states where (1) they have no Senate competition in 2022 so we use their most recent party outcome for the most recent Senate election in 2020 or (2) forecasting and demographic/polling factors strongly indicate a party direction for the Senate results, to the point where no forecasting is necessary and polling is usually sparse. The paper addresses this in more depth as a potential challenge.

In [11]:
null_results = pd.isnull(sen_data["recent-res"])
competitive_states = [i for i, n in enumerate(null_results) if n == True] 
other_states = list(set(list(range(50))) - set(competitive_states))
print(other_states)
print(competitive_states)

sen_data = sen_data.drop(["State"], axis = 1)
sen_train_X = sen_data.iloc[other_states]
sen_train_X = sen_train_X.drop(["recent-res"], axis = 1)
sen_train_y = sen_data[~sen_data['recent-res'].isnull()]['recent-res']
sen_test_X = sen_data.iloc[competitive_states]
sen_test_X = sen_test_X.drop(["recent-res"], axis = 1)

[0, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 29, 30, 31, 33, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 47, 49]
[1, 2, 5, 8, 9, 24, 27, 28, 32, 34, 37, 46, 48]


In [12]:
lrmodel.fit(sen_train_X,sen_train_y.values.ravel())
lassomodel.fit(sen_train_X,sen_train_y.values.ravel())
ridgemodel.fit(sen_train_X,sen_train_y.values.ravel())
elo2model.fit(sen_train_X,sen_train_y.values.ravel())
elo4model.fit(sen_train_X,sen_train_y.values.ravel())
elo6model.fit(sen_train_X,sen_train_y.values.ravel())
elo8model.fit(sen_train_X,sen_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anacond

Pipeline(steps=[('standardize', StandardScaler()),
                ('elo8_reg',
                 LogisticRegression(l1_ratio=0.8, penalty='elasticnet',
                                    solver='saga'))])

In [13]:
## Form predictions over all models
logpreds = lrmodel.predict(sen_test_X)
lassopreds = lassomodel.predict(sen_test_X)
ridgepreds = ridgemodel.predict(sen_test_X)
elo2preds = elo2model.predict(sen_test_X)
elo4preds = elo4model.predict(sen_test_X)
elo6preds = elo6model.predict(sen_test_X)
elo8preds = elo8model.predict(sen_test_X)

The indices above correspond to the following states: [Alaska, Arizona, Colorado, Florida, Georgia, Missouri, Nevada, New Hampshire, North Carolina, Ohio, Pennsylvania, Washington, Wisconsin]

## More Inference for Senate Model

In [14]:
## Coefficients organized from least important to most important
print("Logistic Regression - No Penalty")
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print("LASSO Regression")
print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("Ridge Regression")
print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.2 Logistic Regression")
print(np.sort(elo2r.coef_))
print(np.argsort(np.array(elo2r.coef_)))

print("EL-0.4 Logistic Regression")
print(np.sort(elo4r.coef_))
print(np.argsort(np.array(elo4r.coef_)))

print("EL-0.6 Logistic Regression")
print(np.sort(elo6r.coef_))
print(np.argsort(np.array(elo6r.coef_)))

print("EL-0.8 Logistic Regression")
print(np.sort(elo8r.coef_))
print(np.argsort(np.array(elo8r.coef_)))

Logistic Regression - No Penalty
[[-0.3654782  -0.29566647 -0.2321475  -0.16869273 -0.13088233 -0.11276925
  -0.09329758 -0.06735478 -0.03669928 -0.03510608  0.01228492  0.1019141
   0.12511811  0.12864027  0.14361447  0.15301959  0.16500346  0.17626594
   0.21319514  0.23828687  0.23959686  0.23959686  0.2757649   0.2941162
   0.34679045  0.36928419  0.36961009  0.3850319   0.38684197  0.39895523
   0.42630035  0.45408713  0.45408713  0.49245327  0.50254919  0.55668913
   0.61311397  0.73054915  0.74977644]]
[[27 15 31 17 30 16 37 35  0 20 21 29 25  1 14 18 24 23 26 33 36  6 22 32
  10  2  4 19  3  5  8  7 34 12 38 28 13 11  9]]
LASSO Regression
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.01484742 0.05364336 0.05364336 0.05991064
  0.06327985 0.18464073 0.18581986 0.

## New Data Set

Here we apply a couple of small feature changes and train and test over similar, but new datasets.

In [15]:
pres_data_new = pd.read_excel('full-data.xlsx', sheet_name=2)
sen_data_new = pd.read_excel('full-data.xlsx', sheet_name=3)

In [16]:
## Training Set Info
region_enc = le.fit_transform(pres_data_new['Region'])
pres_data_new["Region"] = region_enc
sen_data_new["Region"] = region_enc
noninc_cols_train = ['State','2020 Population','recent-five-polling-avg-20','polling-party-lead-20','pres2016','pres2020']
pres_train_X = pres_data_new.drop(noninc_cols_train, axis=1)
pres_train_y = pres_data_new[['pres2016']]

In [17]:
## Test Set Info
noninc_cols_test = ['State','2010 Population','recent-five-polling-avg-16','polling-party-lead-16','pres2000','pres2020']
pres_test_X = pres_data_new.drop(noninc_cols_test, axis=1)
pres_test_y = pres_data_new[['pres2020']]
col_move = pres_test_X.pop("pres2016")
pres_test_X.insert(2, "pres2016", col_move)
dict = {'2020 Population': '2010 Population',
        'polling-party-lead-20': 'polling-party-lead-16',
        'pres2016': 'pres2000',
        'recent-five-polling-avg-20': 'recent-five-polling-avg-16',}
pres_test_X.rename(columns=dict,inplace=True)

In [18]:
pres_train_X.head()

,Region,2010 Population,pres2000,pres2004,pres2008,pres2012,gdp-per-capita,high-school-pop,some-college,associates,...,white,inc-party,inc-w-recent,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg-16,polling-party-lead-16,pvi,party-pvi
0,2,4779736,0,0,0,0,49027,30.3,21.6,8.7,...,0.640074,1,1,6829526.55,37491.77,52035,16,0,15,0
1,3,710231,0,0,0,0,75027,28.4,26.0,8.7,...,0.627645,0,1,7527972.46,3566.69,77790,4,1,9,0
2,3,6392017,0,0,0,0,56511,23.8,24.9,8.9,...,0.517834,0,0,13839094.88,38865284.52,61529,2,0,3,0
3,2,2915918,0,0,0,0,47770,33.9,22.0,7.5,...,0.692194,1,1,5973540.80,26852.08,49475,23,0,16,0
4,3,37253956,1,1,1,1,85546,20.4,20.9,8.0,...,0.330164,1,1,135852.28,9467165.58,78672,28,1,14,1


In [19]:
lrmodel.fit(pres_train_X,pres_train_y.values.ravel())
lassomodel.fit(pres_train_X,pres_train_y.values.ravel())
ridgemodel.fit(pres_train_X,pres_train_y.values.ravel())
elo2model.fit(pres_train_X,pres_train_y.values.ravel())
elo4model.fit(pres_train_X,pres_train_y.values.ravel())
elo6model.fit(pres_train_X,pres_train_y.values.ravel())
elo8model.fit(pres_train_X,pres_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anacond

Pipeline(steps=[('standardize', StandardScaler()),
                ('elo8_reg',
                 LogisticRegression(l1_ratio=0.8, penalty='elasticnet',
                                    solver='saga'))])

In [20]:
## Coefficients organized from least important to most important
print("Logistic Regression - No Penalty")
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print("LASSO Regression")
print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("Ridge Regression")
print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.2 Logistic Regression")
print(np.sort(elo2r.coef_))
print(np.argsort(np.array(elo2r.coef_)))

print("EL-0.4 Logistic Regression")
print(np.sort(elo4r.coef_))
print(np.argsort(np.array(elo4r.coef_)))

print("EL-0.6 Logistic Regression")
print(np.sort(elo6r.coef_))
print(np.argsort(np.array(elo6r.coef_)))

print("EL-0.8 Logistic Regression")
print(np.sort(elo8r.coef_))
print(np.argsort(np.array(elo8r.coef_)))

Logistic Regression - No Penalty
[[-0.65816431 -0.6185158  -0.5705371  -0.4941598  -0.24733818 -0.16513896
  -0.14773876  0.02313837  0.07506653  0.1448826   0.14805276  0.14842535
   0.18296132  0.18964226  0.21046986  0.24578449  0.25282397  0.27711688
   0.34074449  0.354171    0.40510053  0.42721982  0.4922806   0.50150591
   0.54859348  0.57446586  0.62947868  0.78865606  2.06390705]]
[[ 7  1 22  9 19 15 13  8 21  0  6 17 27  2  3 16 20 10 14 25 18 24  4 12
  23  5 11 26 28]]
LASSO Regression
[[-0.05557487  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.00534179  0.2245107   0.2425763   0.68304182  2.4817632 ]]
[[ 7  0 25 24 23 22 21 20 19 18 17 16 15 27 14 12 10  9  8  6  3  2  1 13
   4 11  5 26 28]]
Ridge Regression
[[-4.13380778e-01 -3.85599559e-01 -3.72958971e-01 -2.213720

In [21]:
print(pres_train_X.columns[28])
print(pres_train_X.columns[26])
print(pres_train_X.columns[5])
print(pres_train_X.columns[11])
print(pres_train_X.columns[4])
print(pres_train_X.columns[24])

party-pvi
polling-party-lead-16
pres2012
grad-professional
pres2008
median-hh-income


In [22]:
print(pres_train_X.columns[10])
print(pres_train_X.columns[9])
print(pres_train_X.columns[8])
print(pres_train_X.columns[6])
print(pres_train_X.columns[3])
print(pres_train_X.columns[2])

bachelors
associates
some-college
gdp-per-capita
pres2004
pres2000


In [23]:
forest = RandomForestClassifier(n_estimators = 120, random_state = 83)
forest.fit(pres_train_X,pres_train_y.values.ravel())

xgboost1 = xgb.XGBClassifier(base_score=0.5, booster='gbtree', random_state = 83, colsample_bytree = 0.3, learning_rate = 0.1, alpha = 10, n_estimators = 120)
xgboost1.fit(pres_train_X,pres_train_y.values.ravel())

## Feature Importances for Ensembles/Trees
print(np.sort(forest.feature_importances_))
print(np.argsort(np.array(forest.feature_importances_)))
print(np.sort(xgboost1.feature_importances_))
print(np.argsort(np.array(xgboost1.feature_importances_)))

[1.49664751e-04 1.96348605e-03 2.19787194e-03 2.23746898e-03
 4.09094629e-03 4.10388258e-03 5.64675067e-03 6.71188500e-03
 7.35686638e-03 7.45568485e-03 8.70821793e-03 1.11084831e-02
 1.24441818e-02 1.29264470e-02 1.29269990e-02 1.29874984e-02
 1.42689151e-02 1.76963942e-02 3.20600728e-02 3.59874119e-02
 3.91224566e-02 4.79442737e-02 5.10695269e-02 5.36274339e-02
 6.11344545e-02 7.54317723e-02 1.05225719e-01 1.16296323e-01
 2.37118912e-01]
[21 13 15 20  0  9 12 17 25 23  7 19  2  6  8  1 16 27 22 18 10 14  3  5
 24  4 11 26 28]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00884877 0.12220033 0.3481493  0.52080166]
[ 0 25 24 23 22 21 20 19 18 17 16 15 27 14 12 10  9  8  7  6  4  3  2  1
 13 11  5 28 26]


In [24]:
rfpreds = forest.predict(pres_test_X)
xgbpreds = xgboost1.predict(pres_test_X)
print("Random Forests")
print(rfpreds)
print("Gradient Boosting")
print(xgbpreds)

Random Forests
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
Gradient Boosting
[0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]


## Senate Fits

In [25]:
sen_data_new = sen_data_new.drop(["State"], axis = 1)
sen_train_X = sen_data_new.iloc[other_states]
sen_train_X = sen_train_X.drop(["recent-res"], axis = 1)
sen_train_y = sen_data_new[~sen_data['recent-res'].isnull()]['recent-res']
sen_test_X = sen_data_new.iloc[competitive_states]
sen_test_X = sen_test_X.drop(["recent-res"], axis = 1)

In [26]:
sen_data_new.head()

,Region,2020 Population,pres2012,pres2016,pres2020,sen1-recent,sen2-recent,gdp-per-capita,high-school-pop,some-college,...,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg,polling-party-lead,pvi,party-pvi,employment-rate,urban-pct-2010,recent-res
0,2,5024279,0,0,0,0,0,49027,30.3,21.6,...,6829526.55,37491.77,52035,20.4,0,15,0,54.0,59.04,0.0
1,3,733391,0,0,0,0,0,75027,28.4,26.0,...,7527972.46,3566.69,77790,6.5,0,9,0,59.6,66.02,NaN
2,3,7151502,0,0,1,1,1,56511,23.8,24.9,...,13839094.88,38865284.52,61529,7.0,1,3,0,56.2,89.81,NaN
3,2,3011524,0,0,0,0,0,47770,33.9,22.0,...,5973540.80,26852.08,49475,45.0,0,16,0,54.8,56.16,0.0
4,3,39538223,1,1,1,1,1,85546,20.4,20.9,...,135852.28,9467165.58,78672,25.0,1,14,1,59.4,94.95,1.0


In [27]:
lrmodel.fit(sen_train_X,sen_train_y.values.ravel())
lassomodel.fit(sen_train_X,sen_train_y.values.ravel())
ridgemodel.fit(sen_train_X,sen_train_y.values.ravel())
elo2model.fit(sen_train_X,sen_train_y.values.ravel())
elo4model.fit(sen_train_X,sen_train_y.values.ravel())
elo6model.fit(sen_train_X,sen_train_y.values.ravel())
elo8model.fit(sen_train_X,sen_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anacond

Pipeline(steps=[('standardize', StandardScaler()),
                ('elo8_reg',
                 LogisticRegression(l1_ratio=0.8, penalty='elasticnet',
                                    solver='saga'))])

In [28]:
## Coefficients organized from least important to most important
print("Logistic Regression - No Penalty")
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print("LASSO Regression")
print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("Ridge Regression")
print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.2 Logistic Regression")
print(np.sort(elo2r.coef_))
print(np.argsort(np.array(lr.coef_)))

print("EL-0.4 Logistic Regression")
print(np.sort(elo4r.coef_))
print(np.argsort(np.array(lassor.coef_)))

print("EL-0.6 Logistic Regression")
print(np.sort(elo6r.coef_))
print(np.argsort(np.array(ridger.coef_)))

print("EL-0.8 Logistic Regression")
print(np.sort(elo8r.coef_))
print(np.argsort(np.array(lr.coef_)))

Logistic Regression - No Penalty
[[-0.38412377 -0.34350295 -0.29730712 -0.21311376 -0.18346477 -0.16913418
  -0.14138028 -0.10735884 -0.00914383  0.00477844  0.02002939  0.02203835
   0.06319089  0.09239746  0.09791483  0.10356843  0.15329033  0.16856243
   0.246915    0.25000295  0.25653017  0.26765058  0.26765058  0.31604508
   0.49802471  0.64183335  0.70132133  0.73252202  0.76001276  0.76001276
   0.96836232  1.27865803]]
[[24 20  8 28 30 10  0  9 23 13 22 14 18 17 11  1 19  7 15 26 16  3 29 25
  12 31  5  2  4 27 21  6]]
LASSO Regression
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.06923838 0.21555333 0.26984692 0.2820582  0.39402522 0.43515587
  0.43515587 1.58375768]]
[[ 0 29 28 26 25 24 23 22 20 19 18 17 16 30 15 13 11 10  9  8  1  7 14  3
  12  5 31  2 21 27  4  6]]
Ridge Regressi

In [29]:
print(sen_train_X.columns[6])
print(sen_train_X.columns[21])
print(sen_train_X.columns[27])
print(sen_train_X.columns[4])

sen2-recent
inc-party
polling-party-lead
pres2020


In [30]:
forest.fit(sen_train_X,sen_train_y.values.ravel())
xgboost1.fit(sen_train_X,sen_train_y.values.ravel())

logpreds = lrmodel.predict(sen_test_X)
lassopreds = lassomodel.predict(sen_test_X)
ridgepreds = ridgemodel.predict(sen_test_X)
elo2preds = elo2model.predict(sen_test_X)
elo4preds = elo4model.predict(sen_test_X)
elo6preds = elo6model.predict(sen_test_X)
elo8preds = elo8model.predict(sen_test_X)
rfpreds = forest.predict(sen_test_X)
xgbpreds = xgboost1.predict(sen_test_X)

print(logpreds)
print(lassopreds)
print(ridgepreds)
print(elo2preds)
print(elo4preds)
print(elo6preds)
print(elo8preds)
print(rfpreds)
print(xgbpreds)

[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1.]
[0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0.]
[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0.]
[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0]


The indices above correspond to the following states: [Alaska, Arizona, Colorado, Florida, Georgia, Missouri, Nevada, New Hampshire, North Carolina, Ohio, Pennsylvania, Washington, Wisconsin]

In [31]:
print(np.sort(forest.feature_importances_))
print(np.argsort(np.array(forest.feature_importances_)))
print(np.sort(xgboost1.feature_importances_))
print(np.argsort(np.array(xgboost1.feature_importances_)))

[0.         0.         0.00063837 0.00189809 0.00337442 0.00339247
 0.0045115  0.00587309 0.00630449 0.00646727 0.00647933 0.00747065
 0.0076196  0.00903482 0.00909733 0.01174924 0.0126972  0.01469967
 0.01558472 0.01882768 0.02053692 0.03598656 0.03621964 0.04266007
 0.04860869 0.05625146 0.0601764  0.0979208  0.09905442 0.10028748
 0.10743275 0.1491449 ]
[ 1 22 14  0 16 17 10 30 13  7 28 26 19 18 20  8 11  9 24 23 21 15 31 25
  5  3 29  4 27  2 12  6]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1.]
[ 0 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7
  5  4  3  2  1 30 31  6]


In [32]:
print(sen_train_X.columns[6])
print(sen_train_X.columns[12])
print(sen_train_X.columns[2])
print(sen_train_X.columns[27])

print(sen_train_X.columns[5])
print(sen_train_X.columns[29])
print(sen_train_X.columns[27])
print(sen_train_X.columns[6])

sen2-recent
grad-professional
pres2012
polling-party-lead
sen1-recent
party-pvi
polling-party-lead
sen2-recent
